In [18]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [19]:
#declare all repo ids you would like to produce charts for
repo_set = {25440, 25448}

#can be set as 'competitors' or 'repo'
#'competitors' will group graphs by type, so it is easy to compare across repos
# 'repo' will group graphs by repo so it is easy to look at all the contributor data for each repo
display_grouping = 'repo'

#if display_grouping is set to 'competitors', enter the repo ids you do no want to alias, if 'display_grouping' is set to repo the list will not effect anything
not_aliased_repos = [25440, 25448]

begin_date = '2019-10-01'
end_date = '2020-10-31'

#specify number of outliers for removal in scatter plot
scatter_plot_outliers_removed = 5
save_files = False

In [20]:
pr_all = pd.DataFrame()

for repo_id in repo_set: 

    pr_query = salc.sql.text(f"""
                    SELECT
                        repo.repo_id AS repo_id,
                        pull_requests.pr_src_id AS pr_src_id,
                        repo.repo_name AS repo_name,
                        pr_src_author_association,
                        repo_groups.rg_name AS repo_group,
                        pull_requests.pr_src_state,
                        pull_requests.pr_merged_at,
                        pull_requests.pr_created_at AS pr_created_at,
                        pull_requests.pr_closed_at AS pr_closed_at,
                        date_part( 'year', pr_created_at :: DATE ) AS CREATED_YEAR,
                        date_part( 'month', pr_created_at :: DATE ) AS CREATED_MONTH,
                        date_part( 'year', pr_closed_at :: DATE ) AS CLOSED_YEAR,
                        date_part( 'month', pr_closed_at :: DATE ) AS CLOSED_MONTH,
                        pr_src_meta_label,
                        pr_head_or_base,
                        ( EXTRACT ( EPOCH FROM pull_requests.pr_closed_at ) - EXTRACT ( EPOCH FROM pull_requests.pr_created_at ) ) / 3600 AS hours_to_close,
                        ( EXTRACT ( EPOCH FROM pull_requests.pr_closed_at ) - EXTRACT ( EPOCH FROM pull_requests.pr_created_at ) ) / 86400 AS days_to_close, 
                        ( EXTRACT ( EPOCH FROM first_response_time ) - EXTRACT ( EPOCH FROM pull_requests.pr_created_at ) ) / 3600 AS hours_to_first_response,
                        ( EXTRACT ( EPOCH FROM first_response_time ) - EXTRACT ( EPOCH FROM pull_requests.pr_created_at ) ) / 86400 AS days_to_first_response, 
                        ( EXTRACT ( EPOCH FROM last_response_time ) - EXTRACT ( EPOCH FROM pull_requests.pr_created_at ) ) / 3600 AS hours_to_last_response,
                        ( EXTRACT ( EPOCH FROM last_response_time ) - EXTRACT ( EPOCH FROM pull_requests.pr_created_at ) ) / 86400 AS days_to_last_response, 
                        first_response_time,
                        last_response_time,
                        average_time_between_responses,
                        assigned_count,
                        review_requested_count,
                        labeled_count,
                        subscribed_count,
                        mentioned_count,
                        referenced_count,
                        closed_count,
                        head_ref_force_pushed_count,
                        merged_count,
                        milestoned_count,
                        unlabeled_count,
                        head_ref_deleted_count,
                        comment_count,
                        lines_added, 
                        lines_removed,
                        commit_count, 
                        file_count
                    FROM
                        repo,
                        repo_groups,
                        pull_requests LEFT OUTER JOIN ( 
                            SELECT pull_requests.pull_request_id,
                            count(*) FILTER (WHERE action = 'assigned') AS assigned_count,
                            count(*) FILTER (WHERE action = 'review_requested') AS review_requested_count,
                            count(*) FILTER (WHERE action = 'labeled') AS labeled_count,
                            count(*) FILTER (WHERE action = 'unlabeled') AS unlabeled_count,
                            count(*) FILTER (WHERE action = 'subscribed') AS subscribed_count,
                            count(*) FILTER (WHERE action = 'mentioned') AS mentioned_count,
                            count(*) FILTER (WHERE action = 'referenced') AS referenced_count,
                            count(*) FILTER (WHERE action = 'closed') AS closed_count,
                            count(*) FILTER (WHERE action = 'head_ref_force_pushed') AS head_ref_force_pushed_count,
                            count(*) FILTER (WHERE action = 'head_ref_deleted') AS head_ref_deleted_count,
                            count(*) FILTER (WHERE action = 'milestoned') AS milestoned_count,
                            count(*) FILTER (WHERE action = 'merged') AS merged_count,
                            MIN(message.msg_timestamp) AS first_response_time,
                            COUNT(DISTINCT message.msg_timestamp) AS comment_count,
                            MAX(message.msg_timestamp) AS last_response_time,
                            (MAX(message.msg_timestamp) - MIN(message.msg_timestamp)) / COUNT(DISTINCT message.msg_timestamp) AS average_time_between_responses
                            FROM pull_request_events, pull_requests, repo, pull_request_message_ref, message
                            WHERE repo.repo_id = {repo_id}
                            AND repo.repo_id = pull_requests.repo_id
                            AND pull_requests.pull_request_id = pull_request_events.pull_request_id
                            AND pull_requests.pull_request_id = pull_request_message_ref.pull_request_id
                            AND pull_request_message_ref.msg_id = message.msg_id
                            GROUP BY pull_requests.pull_request_id
                        ) response_times
                        ON pull_requests.pull_request_id = response_times.pull_request_id
                        LEFT OUTER JOIN (
                            SELECT pull_request_commits.pull_request_id, count(DISTINCT pr_cmt_sha) AS commit_count                                FROM pull_request_commits, pull_requests, pull_request_meta
                            WHERE pull_requests.pull_request_id = pull_request_commits.pull_request_id
                            AND pull_requests.pull_request_id = pull_request_meta.pull_request_id
                            AND pull_requests.repo_id = {repo_id}
                            AND pr_cmt_sha <> pull_requests.pr_merge_commit_sha
                            AND pr_cmt_sha <> pull_request_meta.pr_sha
                            GROUP BY pull_request_commits.pull_request_id
                        ) all_commit_counts
                        ON pull_requests.pull_request_id = all_commit_counts.pull_request_id
                        LEFT OUTER JOIN (
                            SELECT MAX(pr_repo_meta_id), pull_request_meta.pull_request_id, pr_head_or_base, pr_src_meta_label
                            FROM pull_requests, pull_request_meta
                            WHERE pull_requests.pull_request_id = pull_request_meta.pull_request_id
                            AND pull_requests.repo_id = {repo_id}
                            AND pr_head_or_base = 'base'
                            GROUP BY pull_request_meta.pull_request_id, pr_head_or_base, pr_src_meta_label
                        ) base_labels
                        ON base_labels.pull_request_id = all_commit_counts.pull_request_id
                        LEFT OUTER JOIN (
                            SELECT sum(cmt_added) AS lines_added, sum(cmt_removed) AS lines_removed, pull_request_commits.pull_request_id, count(DISTINCT cmt_filename) AS file_count
                            FROM pull_request_commits, commits, pull_requests, pull_request_meta
                            WHERE cmt_commit_hash = pr_cmt_sha
                            AND pull_requests.pull_request_id = pull_request_commits.pull_request_id
                            AND pull_requests.pull_request_id = pull_request_meta.pull_request_id
                            AND pull_requests.repo_id = {repo_id}
                            AND commits.repo_id = pull_requests.repo_id
                            AND commits.cmt_commit_hash <> pull_requests.pr_merge_commit_sha
                            AND commits.cmt_commit_hash <> pull_request_meta.pr_sha
                            GROUP BY pull_request_commits.pull_request_id
                        ) master_merged_counts 
                        ON base_labels.pull_request_id = master_merged_counts.pull_request_id                    
                    WHERE 
                        repo.repo_group_id = repo_groups.repo_group_id 
                        AND repo.repo_id = pull_requests.repo_id 
                        AND repo.repo_id = {repo_id} 
                    ORDER BY
                       merged_count DESC
        """)
    pr_a = pd.read_sql(pr_query, con=engine)
    if not pr_all.empty: 
        pr_all = pd.concat([pr_all, pr_a]) 
    else: 
        # first repo
        pr_all = pr_a
display(pr_all.head())
pr_all.dtypes

,repo_id,pr_src_id,repo_name,pr_src_author_association,repo_group,pr_src_state,pr_merged_at,pr_created_at,pr_closed_at,created_year,...,head_ref_force_pushed_count,merged_count,milestoned_count,unlabeled_count,head_ref_deleted_count,comment_count,lines_added,lines_removed,commit_count,file_count
0,25440,621733853,augur,NONE,chaoss,closed,NaT,2021-04-23 02:56:58,2021-04-23 02:57:59,2021.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25440,103980067,augur,CONTRIBUTOR,chaoss,closed,2017-01-31 20:09:52,2017-01-31 17:08:16,2017-01-31 20:09:52,2017.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,1.0
2,25440,492966815,augur,MEMBER,chaoss,closed,2020-09-25 10:00:04,2020-09-25 09:59:51,2020-09-25 10:00:04,2020.0,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,28.0,4.0,8.0
3,25440,494794060,augur,MEMBER,chaoss,closed,NaT,2020-09-29 12:29:30,2020-09-29 15:07:00,2020.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,25440,494794806,augur,MEMBER,chaoss,closed,2020-09-29 12:41:54,2020-09-29 12:30:45,2020-09-29 12:41:54,2020.0,...,NaN,NaN,NaN,NaN,NaN,NaN,86.0,15.0,9.0,5.0


repo_id                                     int64
pr_src_id                                   int64
repo_name                                  object
pr_src_author_association                  object
repo_group                                 object
pr_src_state                               object
pr_merged_at                       datetime64[ns]
pr_created_at                      datetime64[ns]
pr_closed_at                       datetime64[ns]
created_year                              float64
created_month                             float64
closed_year                               float64
closed_month                              float64
pr_src_meta_label                          object
pr_head_or_base                            object
hours_to_close                            float64
days_to_close                             float64
hours_to_first_response                   float64
days_to_first_response                    float64
hours_to_last_response                    float64


In [21]:
# change count columns from float datatype to integer
pr_all[['assigned_count',
          'review_requested_count',
          'labeled_count',
          'subscribed_count',
          'mentioned_count',
          'referenced_count',
          'closed_count',
          'head_ref_force_pushed_count',
          'merged_count',
          'milestoned_count',
          'unlabeled_count',
          'head_ref_deleted_count',
          'comment_count',
        'commit_count',
        'file_count',
        'lines_added',
        'lines_removed'
       ]] = pr_all[['assigned_count',
                                      'review_requested_count',
                                      'labeled_count',
                                      'subscribed_count',
                                      'mentioned_count',
                                      'referenced_count',
                                      'closed_count',
                                        'head_ref_force_pushed_count',
                                    'merged_count',
                                      'milestoned_count',          
                                      'unlabeled_count',
                                      'head_ref_deleted_count',
                                      'comment_count',
                                        'commit_count',
                                        'file_count',
                                        'lines_added',
                                        'lines_removed'
                   ]].astype(float)
# Change years to int so that doesn't display as 2019.0 for example
pr_all[[
            'created_year',
           'closed_year']] = pr_all[['created_year',
                                       'closed_year']].fillna(-1).astype(int).astype(str)
pr_all.dtypes

repo_id                                     int64
pr_src_id                                   int64
repo_name                                  object
pr_src_author_association                  object
repo_group                                 object
pr_src_state                               object
pr_merged_at                       datetime64[ns]
pr_created_at                      datetime64[ns]
pr_closed_at                       datetime64[ns]
created_year                               object
created_month                             float64
closed_year                                object
closed_month                              float64
pr_src_meta_label                          object
pr_head_or_base                            object
hours_to_close                            float64
days_to_close                             float64
hours_to_first_response                   float64
days_to_first_response                    float64
hours_to_last_response                    float64


In [22]:
print(pr_all['repo_name'].unique())

['augur' 'grimoirelab']


In [23]:
# Get days for average_time_between_responses time delta

pr_all['average_days_between_responses'] = pr_all['average_time_between_responses'].map(lambda x: x.days).astype(float)
pr_all['average_hours_between_responses'] = pr_all['average_time_between_responses'].map(lambda x: x.days * 24).astype(float)

pr_all.head()

,repo_id,pr_src_id,repo_name,pr_src_author_association,repo_group,pr_src_state,pr_merged_at,pr_created_at,pr_closed_at,created_year,...,milestoned_count,unlabeled_count,head_ref_deleted_count,comment_count,lines_added,lines_removed,commit_count,file_count,average_days_between_responses,average_hours_between_responses
0,25440,621733853,augur,NONE,chaoss,closed,NaT,2021-04-23 02:56:58,2021-04-23 02:57:59,2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25440,103980067,augur,CONTRIBUTOR,chaoss,closed,2017-01-31 20:09:52,2017-01-31 17:08:16,2017-01-31 20:09:52,2017,...,NaN,NaN,NaN,NaN,1.0,0.0,1.0,1.0,NaN,NaN
2,25440,492966815,augur,MEMBER,chaoss,closed,2020-09-25 10:00:04,2020-09-25 09:59:51,2020-09-25 10:00:04,2020,...,NaN,NaN,NaN,NaN,12.0,28.0,4.0,8.0,NaN,NaN
3,25440,494794060,augur,MEMBER,chaoss,closed,NaT,2020-09-29 12:29:30,2020-09-29 15:07:00,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,25440,494794806,augur,MEMBER,chaoss,closed,2020-09-29 12:41:54,2020-09-29 12:30:45,2020-09-29 12:41:54,2020,...,NaN,NaN,NaN,NaN,86.0,15.0,9.0,5.0,NaN,NaN


In [24]:
start_date = pd.to_datetime(begin_date)
# end_date = pd.to_datetime('2020-02-01 09:00:00')
end_date = pd.to_datetime(end_date)
pr_all = pr_all[(pr_all['pr_created_at'] > start_date) & (pr_all['pr_closed_at'] < end_date)]

pr_all['created_year'] = pr_all['created_year'].map(int)
pr_all['created_month'] = pr_all['created_month'].map(int)
pr_all['created_month'] = pr_all['created_month'].map(lambda x: '{0:0>2}'.format(x))
pr_all['created_yearmonth'] = pd.to_datetime(pr_all['created_year'].map(str) + '-' + pr_all['created_month'].map(str) + '-01')
pr_all.head(1)

,repo_id,pr_src_id,repo_name,pr_src_author_association,repo_group,pr_src_state,pr_merged_at,pr_created_at,pr_closed_at,created_year,...,unlabeled_count,head_ref_deleted_count,comment_count,lines_added,lines_removed,commit_count,file_count,average_days_between_responses,average_hours_between_responses,created_yearmonth
2,25440,492966815,augur,MEMBER,chaoss,closed,2020-09-25 10:00:04,2020-09-25 09:59:51,2020-09-25 10:00:04,2020,...,NaN,NaN,NaN,12.0,28.0,4.0,8.0,NaN,NaN,2020-09-01


In [25]:
import datetime
# getting the number of days of (today - created at) for the PRs that are still open
# and putting this in the days_to_close column

# get timedeltas of creation time to todays date/time
days_to_close_open_pr = datetime.datetime.now() - pr_all.loc[pr_all['pr_src_state'] == 'open']['pr_created_at']

# get num days from above timedelta
days_to_close_open_pr = days_to_close_open_pr.apply(lambda x: x.days).astype(int)

# for only OPEN pr's, set the days_to_close column equal to above dataframe
pr_all.loc[pr_all['pr_src_state'] == 'open'] = pr_all.loc[pr_all['pr_src_state'] == 'open'].assign(days_to_close=days_to_close_open_pr)

pr_all.loc[pr_all['pr_src_state'] == 'open'].head()

,repo_id,pr_src_id,repo_name,pr_src_author_association,repo_group,pr_src_state,pr_merged_at,pr_created_at,pr_closed_at,created_year,...,unlabeled_count,head_ref_deleted_count,comment_count,lines_added,lines_removed,commit_count,file_count,average_days_between_responses,average_hours_between_responses,created_yearmonth


In [26]:
# initiate column by setting all null datetimes
pr_all['closed_yearmonth'] = pd.to_datetime(np.nan)

# Fill column with prettified string of year/month closed that looks like: 2019-07-01
pr_all.loc[pr_all['pr_src_state'] == 'closed'] = pr_all.loc[pr_all['pr_src_state'] == 'closed'].assign(
    closed_yearmonth = pd.to_datetime(pr_all.loc[pr_all['pr_src_state'] == 'closed']['closed_year'].astype(int
        ).map(str) + '-' + pr_all.loc[pr_all['pr_src_state'] == 'closed']['closed_month'].astype(int).map(str) + '-01'))

pr_all.loc[pr_all['pr_src_state'] == 'closed']

,repo_id,pr_src_id,repo_name,pr_src_author_association,repo_group,pr_src_state,pr_merged_at,pr_created_at,pr_closed_at,created_year,...,head_ref_deleted_count,comment_count,lines_added,lines_removed,commit_count,file_count,average_days_between_responses,average_hours_between_responses,created_yearmonth,closed_yearmonth
2,25440,492966815,augur,MEMBER,chaoss,closed,2020-09-25 10:00:04,2020-09-25 09:59:51,2020-09-25 10:00:04,2020,...,NaN,NaN,12.0,28.0,4.0,8.0,NaN,NaN,2020-09-01,2020-09-01
3,25440,494794060,augur,MEMBER,chaoss,closed,NaT,2020-09-29 12:29:30,2020-09-29 15:07:00,2020,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2020-09-01,2020-09-01
4,25440,494794806,augur,MEMBER,chaoss,closed,2020-09-29 12:41:54,2020-09-29 12:30:45,2020-09-29 12:41:54,2020,...,NaN,NaN,86.0,15.0,9.0,5.0,NaN,NaN,2020-09-01,2020-09-01
5,25440,494804174,augur,MEMBER,chaoss,closed,2020-09-29 12:45:31,2020-09-29 12:45:08,2020-09-29 12:45:32,2020,...,NaN,NaN,1.0,0.0,1.0,1.0,NaN,NaN,2020-09-01,2020-09-01
6,25440,494907712,augur,MEMBER,chaoss,closed,2020-09-30 21:13:18,2020-09-29 15:08:00,2020-09-30 21:13:18,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-01,2020-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,25448,365584359,grimoirelab,NONE,chaoss,closed,NaT,2020-01-21 23:36:42,2020-01-21 23:40:10,2020,...,1.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,2020-01-01,2020-01-01
163,25448,365607031,grimoirelab,NONE,chaoss,closed,NaT,2020-01-22 00:42:57,2020-02-04 15:15:00,2020,...,0.0,4.0,NaN,NaN,NaN,NaN,3.0,72.0,2020-01-01,2020-02-01
164,25448,356138303,grimoirelab,MEMBER,chaoss,closed,NaT,2019-12-23 00:51:55,2020-02-10 16:46:32,2019,...,0.0,1.0,NaN,NaN,1.0,NaN,0.0,0.0,2019-12-01,2020-02-01
167,25448,402822645,grimoirelab,CONTRIBUTOR,chaoss,closed,NaT,2020-04-13 20:22:36,2020-06-03 15:07:12,2020,...,1.0,1.0,NaN,NaN,1.0,NaN,0.0,0.0,2020-04-01,2020-06-01


In [27]:
""" Merged flag """
if 'pr_merged_at' in pr_all.columns.values:
    pr_all['pr_merged_at'] = pr_all['pr_merged_at'].fillna(0)
    pr_all['merged_flag'] = 'Not Merged / Rejected'
    pr_all['merged_flag'].loc[pr_all['pr_merged_at'] != 0] = 'Merged / Accepted'
    pr_all['merged_flag'].loc[pr_all['pr_src_state'] == 'open'] = 'Still Open'
    del pr_all['pr_merged_at']
pr_all['merged_flag']

2          Merged / Accepted
3      Not Merged / Rejected
4          Merged / Accepted
5          Merged / Accepted
6          Merged / Accepted
               ...          
162    Not Merged / Rejected
163    Not Merged / Rejected
164    Not Merged / Rejected
167    Not Merged / Rejected
172    Not Merged / Rejected
Name: merged_flag, Length: 543, dtype: object

In [28]:
# Isolate the different state PRs for now
pr_open = pr_all.loc[pr_all['pr_src_state'] == 'open']
pr_closed = pr_all.loc[pr_all['pr_src_state'] == 'closed']
pr_merged = pr_all.loc[pr_all['merged_flag'] == 'Merged / Accepted']
pr_not_merged = pr_all.loc[pr_all['merged_flag'] == 'Not Merged / Rejected']
pr_closed['merged_flag']

2          Merged / Accepted
3      Not Merged / Rejected
4          Merged / Accepted
5          Merged / Accepted
6          Merged / Accepted
               ...          
162    Not Merged / Rejected
163    Not Merged / Rejected
164    Not Merged / Rejected
167    Not Merged / Rejected
172    Not Merged / Rejected
Name: merged_flag, Length: 543, dtype: object

In [29]:
# Filtering the 80th percentile slowest PRs

def filter_20_per_slowest(input_df):
    pr_slow20_filtered = pd.DataFrame()
    pr_slow20_x = pd.DataFrame()
    for value in repo_set: 
        if not pr_slow20_filtered.empty: 
            pr_slow20x = input_df.query('repo_id==@value')
            pr_slow20x['percentile_rank_local'] = pr_slow20x.days_to_close.rank(pct=True)
            pr_slow20x = pr_slow20x.query('percentile_rank_local >= .8', )
            pr_slow20_filtered = pd.concat([pr_slow20x, pr_slow20_filtered]) 
            reponame = str(value)
            filename = ''.join(['output/pr_slowest20pct', reponame, '.csv'])
            pr_slow20x.to_csv(filename)
        else: 
            # first time
            pr_slow20_filtered = input_df.copy()
            pr_slow20_filtered['percentile_rank_local'] = pr_slow20_filtered.days_to_close.rank(pct=True)
            pr_slow20_filtered = pr_slow20_filtered.query('percentile_rank_local >= .8', )
#     print(pr_slow20_filtered.describe())
    return pr_slow20_filtered

pr_slow20_open = filter_20_per_slowest(pr_open)
pr_slow20_closed = filter_20_per_slowest(pr_closed)
pr_slow20_merged = filter_20_per_slowest(pr_merged)
pr_slow20_not_merged = filter_20_per_slowest(pr_not_merged)
pr_slow20_all = filter_20_per_slowest(pr_all)
pr_slow20_merged#.head()

,repo_id,pr_src_id,repo_name,pr_src_author_association,repo_group,pr_src_state,pr_created_at,pr_closed_at,created_year,created_month,...,lines_added,lines_removed,commit_count,file_count,average_days_between_responses,average_hours_between_responses,created_yearmonth,closed_yearmonth,merged_flag,percentile_rank_local
0,25448,355402007,grimoirelab,MEMBER,chaoss,closed,2019-12-19 22:39:01,2020-01-25 10:00:57,2019,12,...,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-01,2020-01-01,Merged / Accepted,0.934783
17,25448,346800015,grimoirelab,MEMBER,chaoss,closed,2019-11-28 17:57:14,2019-12-04 08:32:13,2019,11,...,NaN,NaN,1.0,NaN,NaN,NaN,2019-11-01,2019-12-01,Merged / Accepted,0.826087
23,25448,355401820,grimoirelab,MEMBER,chaoss,closed,2019-12-19 22:38:43,2020-01-25 10:01:08,2019,12,...,NaN,NaN,1.0,NaN,NaN,NaN,2019-12-01,2020-01-01,Merged / Accepted,0.956522
116,25448,334530507,grimoirelab,MEMBER,chaoss,closed,2019-10-30 21:37:16,2019-12-16 06:57:23,2019,10,...,NaN,NaN,3.0,NaN,0.0,0.0,2019-10-01,2019-12-01,Merged / Accepted,0.978261
117,25448,422295469,grimoirelab,MEMBER,chaoss,closed,2020-05-23 16:21:46,2020-06-03 21:00:21,2020,05,...,NaN,NaN,1.0,NaN,1.0,24.0,2020-05-01,2020-06-01,Merged / Accepted,0.869565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,25448,450459698,grimoirelab,CONTRIBUTOR,chaoss,closed,2020-07-16 19:43:06,2020-07-26 09:27:53,2020,07,...,4347.0,120.0,2.0,40.0,0.0,0.0,2020-07-01,2020-07-01,Merged / Accepted,0.928726
144,25448,493156462,grimoirelab,CONTRIBUTOR,chaoss,closed,2020-09-25 15:41:15,2020-10-07 18:08:10,2020,09,...,27255.0,34.0,3.0,9.0,0.0,0.0,2020-09-01,2020-10-01,Merged / Accepted,0.954644
152,25448,359257618,grimoirelab,CONTRIBUTOR,chaoss,closed,2020-01-04 23:55:54,2020-01-16 13:37:29,2020,01,...,NaN,NaN,NaN,NaN,0.0,0.0,2020-01-01,2020-01-01,Merged / Accepted,0.952484
154,25448,371977090,grimoirelab,MEMBER,chaoss,closed,2020-02-06 15:57:49,2020-02-10 09:52:38,2020,02,...,NaN,NaN,1.0,NaN,0.0,0.0,2020-02-01,2020-02-01,Merged / Accepted,0.825054


In [30]:
#create dictionairy with number as the key and a letter as the value
#this is used to alias repos when using 'compeitor' display grouping
letters = []
nums = []
alpha = 'a'
for i in range(0, 26): 
    letters.append(alpha) 
    alpha = chr(ord(alpha) + 1)
    nums.append(i)
letters = [x.upper() for x in letters]

#create dict out of list of numbers and letters
repo_alias_dict = {nums[i]: letters[i] for i in range(len(nums))}

# create dict in the form {repo_id : repo_name}
aliased_repos = []
repo_dict = {}
count = 0
for repo_id in repo_set:
    
    #find corresponding repo name from each repo_id 
    repo_name = pr_all.loc[pr_all['repo_id'] == repo_id].iloc[0]['repo_name']
    
    #if competitor grouping is enabled turn all repo names, other than the ones in the 'not_aliased_repos' into an alias
    if display_grouping == 'competitors' and not repo_id in not_aliased_repos:
        repo_name =  'Repo ' + repo_alias_dict[count]
        
        #add repo_id to list of aliased repos, this is used for ordering
        aliased_repos.append(repo_id)
        count += 1
        
    #add repo_id and repo names as key value pairs into a dict, this is used to label the title of the visualizations
    repo_dict.update({repo_id : repo_name})

#gurantees that the non_aliased repos come first when display grouping is set as 'competitors'
repo_list = not_aliased_repos + aliased_repos

# Start new visualizations

In [31]:
from bokeh.palettes import Colorblind, mpl, Category20
from bokeh.layouts import gridplot
from bokeh.models.annotations import Title
from bokeh.io import export_png
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, Legend, LabelSet, Range1d, LinearAxis, Label
from bokeh.plotting import figure
from bokeh.models.glyphs import Rect
from bokeh.transform import dodge

try:
    colors = Colorblind[len(repo_set)]
except:
    colors = Colorblind[3]
#mpl['Plasma'][len(repo_set)]
#['A6CEE3','B2DF8A','33A02C','FB9A99']

def remove_outliers(input_df, field, num_outliers_repo_map):
    df_no_outliers = input_df.copy()
    for repo_name, num_outliers in num_outliers_repo_map.items():
        indices_to_drop = input_df.loc[input_df['repo_name'] == repo_name].nlargest(num_outliers, field).index
        df_no_outliers = df_no_outliers.drop(index=indices_to_drop)
    return df_no_outliers


In [32]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import datetime as dt

def visualize_mean_days_to_close(input_df, x_axis='closed_yearmonth', description='Closed', save_file=False, num_remove_outliers=0, drop_outliers_repo=None):

    # Set the df you want to build the viz's for
    driver_df = input_df.copy()
    
    driver_df = driver_df[['repo_id', 'repo_name', 'pr_src_id', 'created_yearmonth', 'closed_yearmonth', 'days_to_close']]

    if save_file:
        driver_df.to_csv('output/c.westw20small {}.csv'.format(description))
    
    driver_df_mean = driver_df.groupby(['repo_id', x_axis, 'repo_name'],as_index=False).mean()
        
    # Total PRS Closed
    fig, ax = plt.subplots()
    # the size of A4 paper
    fig.set_size_inches(16, 8)
    plotter = sns.lineplot(x=x_axis, y='days_to_close', style='repo_name', data=driver_df_mean, sort=True, legend='full', linewidth=2.5, hue='repo_name').set_title("Average Days to Close of {} Pull Requests, July 2017-January 2020".format(description))  
    if save_file:
        fig.savefig('images/slow_20_mean {}.png'.format(description))
    
    # Copying array and deleting the outlier in the copy to re-visualize
    def drop_n_largest(input_df, n, repo_name):
        input_df_copy = input_df.copy()
        indices_to_drop = input_df.loc[input_df['repo_name'] == 'amazon-freertos'].nlargest(n,'days_to_close').index
        print("Indices to drop: {}".format(indices_to_drop))
        input_df_copy = input_df_copy.drop(index=indices_to_drop)
        input_df_copy.loc[input_df['repo_name'] == repo_name]
        return input_df_copy

    if num_remove_outliers > 0 and drop_outliers_repo:
        driver_df_mean_no_outliers = drop_n_largest(driver_df_mean, num_remove_outliers, drop_outliers_repo)
    
        # Total PRS Closed without outlier
        fig, ax = plt.subplots()
        # the size of A4 paper
        fig.set_size_inches(16, 8)
        plotter = sns.lineplot(x=x_axis, y='days_to_close', style='repo_name', data=driver_df_mean_no_outliers, sort=False, legend='full', linewidth=2.5, hue='repo_name').set_title("Average Days to Close among {} Pull Requests Without Outlier, July 2017-January 2020".format(description))
        plotterlabels = ax.set_xticklabels(driver_df_mean_no_outliers[x_axis], rotation=90, fontsize=8)
        if save_file:
            fig.savefig('images/slow_20_mean_no_outlier {}.png'.format(description))
    